# ETL Datos Secundarios

Este notebook procesa los datos secundarios para:
1. **DeudaExternaTotal** (Banco Mundial) - Filtrar México y pivotar de horizontal a vertical
2. **PIB** (Banco Mundial) - Filtrar México y pivotar de horizontal a vertical
3. **DeudaExternaFederal** (Banxico) - Extraer indicadores para comparación con el modelo

Los resultados se guardan en `Datos_Resultado/DatasetSecundarioFinal/`

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Rutas
PATH_BASE = os.getcwd()
PATH_SECUNDARIOS = os.path.join(PATH_BASE, 'Datos_Origen', 'DatosSecundarios')
PATH_OUTPUT = os.path.join(PATH_BASE, 'Datos_Resultado', 'DatasetSecundarioFinal')

# Crear directorio de salida si no existe
os.makedirs(PATH_OUTPUT, exist_ok=True)

print(f"Path base: {PATH_BASE}")
print(f"Path secundarios: {PATH_SECUNDARIOS}")
print(f"Path output: {PATH_OUTPUT}")

Path base: C:\Users\ARTUROJF\Desktop\Final
Path secundarios: C:\Users\ARTUROJF\Desktop\Final\Datos_Origen\DatosSecundarios
Path output: C:\Users\ARTUROJF\Desktop\Final\Datos_Resultado\DatasetSecundarioFinal


## 1. Procesamiento de Deuda Externa Total (Banco Mundial)

Datos del Banco Mundial con indicador: `DT.DOD.DECT.CD` - Deuda externa acumulada total en USD a precios actuales.

**Transformaciones:**
- Filtrar solo México
- Pivotar de formato horizontal (años como columnas) a vertical (años como filas)

In [2]:
# Cargar datos de Deuda Externa Total
file_deuda = os.path.join(PATH_SECUNDARIOS, 'DeudaExternaTotal', 'API_DT.DOD.DECT.CD_DS2_es_csv_v2_32703.csv')
df_deuda_raw = pd.read_csv(file_deuda, skiprows=4)

print(f"Shape original: {df_deuda_raw.shape}")
print(f"Columnas: {df_deuda_raw.columns.tolist()[:10]}...")
df_deuda_raw.head(3)

Shape original: (266, 70)
Columnas: ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965']...


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AFE,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afganistán,AFG,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.596050e+09,2.751987e+09,2.678760e+09,2.661686e+09,3.040072e+09,3.555784e+09,3.393247e+09,3.428117e+09,NaN,NaN


In [3]:
# Filtrar solo México
df_mexico_deuda = df_deuda_raw[df_deuda_raw['Country Name'].str.contains('M.x', case=False, na=False)].copy()
print(f"Registros de México encontrados: {len(df_mexico_deuda)}")
print(f"País: {df_mexico_deuda['Country Name'].values[0]}")
print(f"Indicador: {df_mexico_deuda['Indicator Name'].values[0]}")

Registros de México encontrados: 1
País: México
Indicador: Deuda externa acumulada, total (DOD, US$ a precios actuales)


In [4]:
# Pivotar de horizontal a vertical
# Identificar columnas de años (numéricas)
year_columns = [col for col in df_mexico_deuda.columns if col.isdigit()]
print(f"Años disponibles: {year_columns[:5]}...{year_columns[-5:]}")

# Crear DataFrame pivotado
df_deuda_vertical = df_mexico_deuda.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
    value_vars=year_columns,
    var_name='Anio',
    value_name='Deuda_Externa_Total_USD'
)

# Convertir año a entero
df_deuda_vertical['Anio'] = df_deuda_vertical['Anio'].astype(int)

# Eliminar filas sin datos
df_deuda_vertical = df_deuda_vertical.dropna(subset=['Deuda_Externa_Total_USD'])

# Ordenar por año
df_deuda_vertical = df_deuda_vertical.sort_values('Anio').reset_index(drop=True)

print(f"\nShape después de pivotar: {df_deuda_vertical.shape}")
print(f"Rango de años con datos: {df_deuda_vertical['Anio'].min()} - {df_deuda_vertical['Anio'].max()}")
df_deuda_vertical.head(10)

Años disponibles: ['1960', '1961', '1962', '1963', '1964']...['2020', '2021', '2022', '2023', '2024']

Shape después de pivotar: (54, 6)
Rango de años con datos: 1970 - 2023


,Country Name,Country Code,Indicator Name,Indicator Code,Anio,Deuda_Externa_Total_USD
0,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1970,7.097272e+09
1,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1971,7.627450e+09
2,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1972,8.351505e+09
3,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1973,1.066713e+10
4,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1974,1.408054e+10
5,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1975,1.838141e+10
6,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1976,2.413911e+10
7,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1977,3.138239e+10
8,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1978,3.590534e+10
9,México,MEX,"Deuda externa acumulada, total (DOD, US$ a pre...",DT.DOD.DECT.CD,1979,4.297445e+10


In [5]:
# Simplificar columnas para el archivo final
df_deuda_final = df_deuda_vertical[['Anio', 'Deuda_Externa_Total_USD']].copy()

# Convertir a millones de USD para mejor legibilidad
df_deuda_final['Deuda_Externa_Total_MUSD'] = df_deuda_final['Deuda_Externa_Total_USD'] / 1_000_000

print("Datos finales de Deuda Externa Total (México):")
df_deuda_final.tail(10)

Datos finales de Deuda Externa Total (México):


,Anio,Deuda_Externa_Total_USD,Deuda_Externa_Total_MUSD
44,2014,5.441670e+11,544166.999709
45,2015,5.380152e+11,538015.200056
46,2016,5.447925e+11,544792.453300
47,2017,5.786384e+11,578638.429805
48,2018,6.020007e+11,602000.702060
49,2019,6.174500e+11,617449.991504
50,2020,6.166864e+11,616686.384395
51,2021,6.014741e+11,601474.054898
52,2022,5.854642e+11,585464.239733
53,2023,5.959177e+11,595917.670218


## 2. Procesamiento de PIB (Banco Mundial)

Datos del Banco Mundial con indicador: `NY.GDP.MKTP.CD` - PIB en USD a precios actuales.

**Transformaciones:**
- Filtrar solo México
- Pivotar de formato horizontal a vertical

In [6]:
# Cargar datos de PIB
file_pib = os.path.join(PATH_SECUNDARIOS, 'PIB', 'API_NY.GDP.MKTP.CD_DS2_es_csv_v2_9056.csv')
df_pib_raw = pd.read_csv(file_pib, skiprows=4)

print(f"Shape original: {df_pib_raw.shape}")
print(f"Columnas: {df_pib_raw.columns.tolist()[:10]}...")
df_pib_raw.head(3)

Shape original: (266, 70)
Columnas: ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965']...


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09,3.648573e+09,NaN,NaN
1,NaN,AFE,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,2.420993e+10,2.496326e+10,2.707802e+10,3.177483e+10,3.028492e+10,3.381219e+10,...,8.289612e+11,9.730251e+11,1.012291e+12,1.009747e+12,9.334072e+11,1.085605e+12,1.191639e+12,1.133818e+12,1.205974e+12,NaN
2,Afganistán,AFG,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10,1.715223e+10,NaN,NaN


In [7]:
# Filtrar solo México
df_mexico_pib = df_pib_raw[df_pib_raw['Country Name'].str.contains('M.x', case=False, na=False)].copy()
print(f"Registros de México encontrados: {len(df_mexico_pib)}")
print(f"País: {df_mexico_pib['Country Name'].values[0]}")
print(f"Indicador: {df_mexico_pib['Indicator Name'].values[0]}")

Registros de México encontrados: 1
País: México
Indicador: PIB (US$ a precios actuales)


In [8]:
# Pivotar de horizontal a vertical
year_columns_pib = [col for col in df_mexico_pib.columns if col.isdigit()]
print(f"Años disponibles: {year_columns_pib[:5]}...{year_columns_pib[-5:]}")

# Crear DataFrame pivotado
df_pib_vertical = df_mexico_pib.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
    value_vars=year_columns_pib,
    var_name='Anio',
    value_name='PIB_USD'
)

# Convertir año a entero
df_pib_vertical['Anio'] = df_pib_vertical['Anio'].astype(int)

# Eliminar filas sin datos
df_pib_vertical = df_pib_vertical.dropna(subset=['PIB_USD'])

# Ordenar por año
df_pib_vertical = df_pib_vertical.sort_values('Anio').reset_index(drop=True)

print(f"\nShape después de pivotar: {df_pib_vertical.shape}")
print(f"Rango de años con datos: {df_pib_vertical['Anio'].min()} - {df_pib_vertical['Anio'].max()}")
df_pib_vertical.head(10)

Años disponibles: ['1960', '1961', '1962', '1963', '1964']...['2020', '2021', '2022', '2023', '2024']

Shape después de pivotar: (65, 6)
Rango de años con datos: 1960 - 2024


,Country Name,Country Code,Indicator Name,Indicator Code,Anio,PIB_USD
0,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1960,1.304000e+10
1,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1961,1.416000e+10
2,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1962,1.520000e+10
3,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1963,1.696000e+10
4,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1964,2.008000e+10
5,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1965,2.184000e+10
6,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1966,2.432000e+10
7,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1967,2.656000e+10
8,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1968,2.936000e+10
9,México,MEX,PIB (US$ a precios actuales),NY.GDP.MKTP.CD,1969,3.248000e+10


In [9]:
# Simplificar columnas para el archivo final
df_pib_final = df_pib_vertical[['Anio', 'PIB_USD']].copy()

# Convertir a millones de USD
df_pib_final['PIB_MUSD'] = df_pib_final['PIB_USD'] / 1_000_000

print("Datos finales de PIB (México):")
df_pib_final.tail(10)

Datos finales de PIB (México):


,Anio,PIB_USD,PIB_MUSD
55,2015,1.213294e+12,1.213294e+06
56,2016,1.112233e+12,1.112233e+06
57,2017,1.190721e+12,1.190721e+06
58,2018,1.256300e+12,1.256300e+06
59,2019,1.304106e+12,1.304106e+06
60,2020,1.121065e+12,1.121065e+06
61,2021,1.316569e+12,1.316569e+06
62,2022,1.466465e+12,1.466465e+06
63,2023,1.793799e+12,1.793799e+06
64,2024,1.852723e+12,1.852723e+06


## 3. Procesamiento de Deuda Externa Federal (Banxico)

Datos de Banxico con la posición de deuda externa bruta pública y privada de México.

**Transformaciones:**
- Extraer indicadores principales para comparación con el modelo
- Formato trimestral (ya viene en formato vertical)

In [10]:
# Cargar datos de Deuda Externa Federal (Banxico)
file_federal = os.path.join(PATH_SECUNDARIOS, 'DeudaExternaFederal', 'Consulta_20251204-081823859.xlsx')
df_federal_raw = pd.read_excel(file_federal, header=None)

print(f"Shape original: {df_federal_raw.shape}")

# Mostrar estructura
print("\nEncabezados del archivo:")
print(df_federal_raw.iloc[0:5, 0].tolist())

Shape original: (84, 177)

Encabezados del archivo:
['Banco de México', nan, 'Balanza de pagos', 'Posición de Deuda Externa Bruta. Deuda Pública y Privada', nan]


In [11]:
# Extraer títulos de las columnas (fila 9)
titulos = df_federal_raw.iloc[9, :].tolist()

# Mostrar indicadores disponibles (primeras columnas)
print("Indicadores disponibles (primeras columnas):")
for i, titulo in enumerate(titulos[:10]):
    if pd.notna(titulo):
        nombre_corto = str(titulo).replace('Posición de Deuda Externa Bruta / ', '')
        print(f"{i}: {nombre_corto[:80]}..." if len(str(nombre_corto)) > 80 else f"{i}: {nombre_corto}")

Indicadores disponibles (primeras columnas):
0: Título
1: Total
2: Deuda Externa del Sector Público
3: Deuda Externa del Sector Público / Sector Público Federal (Metodología Tradicion...
4: Deuda Externa del Sector Público / Sector Público Federal (Metodología Tradicion...
5: Deuda Externa del Sector Público / Sector Público Federal (Metodología Tradicion...
6: Deuda Externa del Sector Público / Sector Público Federal (Metodología Tradicion...
7: Deuda Externa del Sector Público / Sector Público Federal (Metodología Tradicion...
8: Deuda Externa del Sector Público / Sector Público Federal (Metodología Tradicion...
9: Deuda Externa del Sector Público / Sector Público Federal (Metodología Tradicion...


In [12]:
# Extraer datos - seleccionar columnas por posición para evitar duplicados
# Columna 0: Fecha
# Columna 1: Total Deuda Externa Bruta
# Columna 2: Sector Público
# Columna 3: Sector Público Federal

idx_fecha = 0
idx_total = 1
idx_sector_publico = 2
idx_federal = 3

# Crear DataFrame solo con columnas necesarias
df_federal = df_federal_raw.iloc[17:, [idx_fecha, idx_total, idx_sector_publico, idx_federal]].copy()
df_federal.columns = ['Fecha', 'Deuda_Externa_Total_MUSD', 'Deuda_Sector_Publico_MUSD', 'Deuda_Publica_Federal_MUSD']
df_federal = df_federal.reset_index(drop=True)

# Convertir fecha a datetime
df_federal['Fecha'] = pd.to_datetime(df_federal['Fecha'], errors='coerce')
df_federal = df_federal.dropna(subset=['Fecha'])

# Convertir columnas numéricas
for col in ['Deuda_Externa_Total_MUSD', 'Deuda_Sector_Publico_MUSD', 'Deuda_Publica_Federal_MUSD']:
    df_federal[col] = pd.to_numeric(df_federal[col], errors='coerce')

print(f"Shape: {df_federal.shape}")
print(f"Rango de fechas: {df_federal['Fecha'].min()} - {df_federal['Fecha'].max()}")
df_federal.head()

Shape: (66, 4)
Rango de fechas: 2009-01-01 00:00:00 - 2025-04-01 00:00:00


,Fecha,Deuda_Externa_Total_MUSD,Deuda_Sector_Publico_MUSD,Deuda_Publica_Federal_MUSD
1,2009-01-01,229185.07,105688.76,85420.12
2,2009-04-01,231762.59,110194.42,86082.42
3,2009-07-01,244967.96,121650.23,91095.42
4,2009-10-01,251486.09,129050.06,96353.70
5,2010-01-01,267190.39,134395.85,96773.00


In [13]:
# Agregar columnas de período
df_federal['Anio'] = df_federal['Fecha'].dt.year
df_federal['Trimestre'] = df_federal['Fecha'].dt.quarter
df_federal['Periodo'] = df_federal['Anio'].astype(str) + '-Q' + df_federal['Trimestre'].astype(str)

# Reordenar columnas
df_federal_final = df_federal[['Fecha', 'Anio', 'Trimestre', 'Periodo',
                                'Deuda_Externa_Total_MUSD', 'Deuda_Sector_Publico_MUSD',
                                'Deuda_Publica_Federal_MUSD']].copy()

print(f"Shape final: {df_federal_final.shape}")
print(f"Columnas: {df_federal_final.columns.tolist()}")
df_federal_final.tail(10)

Shape final: (66, 7)
Columnas: ['Fecha', 'Anio', 'Trimestre', 'Periodo', 'Deuda_Externa_Total_MUSD', 'Deuda_Sector_Publico_MUSD', 'Deuda_Publica_Federal_MUSD']


,Fecha,Anio,Trimestre,Periodo,Deuda_Externa_Total_MUSD,Deuda_Sector_Publico_MUSD,Deuda_Publica_Federal_MUSD
57,2023-01-01,2023,1,2023-Q1,594371.76,329327.02,221951.32
58,2023-04-01,2023,2,2023-Q2,596660.52,329756.01,220488.02
59,2023-07-01,2023,3,2023-Q3,583043.08,319731.49,215250.22
60,2023-10-01,2023,4,2023-Q4,594002.57,331931.43,215107.82
61,2024-01-01,2024,1,2024-Q1,609403.14,339896.94,222028.30
62,2024-04-01,2024,2,2024-Q2,590640.37,324521.21,219009.80
63,2024-07-01,2024,3,2024-Q3,598637.76,328984.04,224748.50
64,2024-10-01,2024,4,2024-Q4,590136.08,315551.39,221048.00
65,2025-01-01,2025,1,2025-Q1,618661.85,341870.43,239987.90
66,2025-04-01,2025,2,2025-Q2,633750.93,346296.09,240253.40


## 4. Guardar Resultados

In [14]:
# Guardar archivos individuales

# 1. Deuda Externa Total (Banco Mundial) - México
file_out_deuda = os.path.join(PATH_OUTPUT, 'DeudaExternaTotal_BancoMundial_Mexico.csv')
df_deuda_final.to_csv(file_out_deuda, index=False, encoding='utf-8-sig')
print(f"Guardado: {file_out_deuda}")

# 2. PIB Corriente (Banco Mundial) - México
file_out_pib = os.path.join(PATH_OUTPUT, 'PIB_Corriente_BancoMundial_Mexico.csv')
df_pib_final.to_csv(file_out_pib, index=False, encoding='utf-8-sig')
print(f"Guardado: {file_out_pib}")

# 3. Deuda Externa Federal (Banxico) - Trimestral
file_out_federal = os.path.join(PATH_OUTPUT, 'DeudaExterna_Banxico_Trimestral.csv')
df_federal_final.to_csv(file_out_federal, index=False, encoding='utf-8-sig')
print(f"Guardado: {file_out_federal}")

print("\n" + "="*60)
print("ETL COMPLETADO EXITOSAMENTE")
print("="*60)

Guardado: C:\Users\ARTUROJF\Desktop\Final\Datos_Resultado\DatasetSecundarioFinal\DeudaExternaTotal_BancoMundial_Mexico.csv
Guardado: C:\Users\ARTUROJF\Desktop\Final\Datos_Resultado\DatasetSecundarioFinal\PIB_Corriente_BancoMundial_Mexico.csv
Guardado: C:\Users\ARTUROJF\Desktop\Final\Datos_Resultado\DatasetSecundarioFinal\DeudaExterna_Banxico_Trimestral.csv

ETL COMPLETADO EXITOSAMENTE


## 5. Resumen de Resultados

In [15]:
# Resumen de archivos generados
print("ARCHIVOS GENERADOS:")
print("="*60)

archivos = os.listdir(PATH_OUTPUT)
for archivo in archivos:
    filepath = os.path.join(PATH_OUTPUT, archivo)
    size = os.path.getsize(filepath) / 1024  # KB
    print(f"  - {archivo} ({size:.1f} KB)")

print("\n" + "="*60)
print("RESUMEN DE DATOS:")
print("="*60)

print(f"\n1. Deuda Externa (Banco Mundial):")
print(f"   - Registros: {len(df_deuda_final)}")
print(f"   - Período: {df_deuda_final['Anio'].min()} - {df_deuda_final['Anio'].max()}")
print(f"   - Último valor: {df_deuda_final['Deuda_Externa_Total_MUSD'].iloc[-1]:,.0f} MUSD")

print(f"\n2. PIB (Banco Mundial):")
print(f"   - Registros: {len(df_pib_final)}")
print(f"   - Período: {df_pib_final['Anio'].min()} - {df_pib_final['Anio'].max()}")
print(f"   - Último valor: {df_pib_final['PIB_MUSD'].iloc[-1]:,.0f} MUSD")

print(f"\n3. Deuda Externa Federal (Banxico):")
print(f"   - Registros trimestrales: {len(df_federal_final)}")
print(f"   - Período: {df_federal_final['Periodo'].iloc[0]} - {df_federal_final['Periodo'].iloc[-1]}")
print(f"   - Último valor total: {df_federal_final['Deuda_Externa_Total_MUSD'].iloc[-1]:,.0f} MUSD")

ARCHIVOS GENERADOS:
  - DeudaExternaTotal_BancoMundial_Mexico.csv (2.0 KB)
  - DeudaExterna_Banxico_Trimestral.csv (3.7 KB)
  - PIB_Corriente_BancoMundial_Mexico.csv (2.4 KB)

RESUMEN DE DATOS:

1. Deuda Externa (Banco Mundial):
   - Registros: 54
   - Período: 1970 - 2023
   - Último valor: 595,918 MUSD

2. PIB (Banco Mundial):
   - Registros: 65
   - Período: 1960 - 2024
   - Último valor: 1,852,723 MUSD

3. Deuda Externa Federal (Banxico):
   - Registros trimestrales: 66
   - Período: 2009-Q1 - 2025-Q2
   - Último valor total: 633,751 MUSD
